In [1]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import keys
import json
import datetime

In [2]:
url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'

In [24]:
size = '200' #maxsize
page = 1

In [25]:
req = requests.get(f'{url}&apikey={keys.ticketmaster}&size={size}&page={page}')

In [423]:
x = json.loads(req.text)

In [424]:
df = pd.DataFrame(x['_embedded']['events'])
df.columns
df.shape

(200, 22)

In [425]:
df.drop(columns = ['id','images','pleaseNote','test','info', 'accessibility', 'locale', 'promoter',
                   'type','_links', 'seatmap', 'products', 'dates', 'outlets', 'promoters'],inplace=True)

In [426]:
df.head()

,name,url,sales,classifications,priceRanges,ticketLimit,_embedded
0,New York Jets vs. Cleveland Browns,https://www.ticketmaster.com/new-york-jets-vs-...,{'public': {'startDateTime': '2019-04-22T14:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","[{'type': 'standard', 'currency': 'USD', 'min'...",{'info': 'There is an overall (6) six ticket l...,"{'venues': [{'name': 'MetLife Stadium', 'type'..."
1,Chicago Bears vs. Los Angeles Chargers,https://www.ticketmaster.com/chicago-bears-vs-...,{'public': {'startDateTime': '2019-04-18T01:30...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","[{'type': 'standard', 'currency': 'USD', 'min'...",{'info': 'There is a fifteen (15) ticket limit.'},"{'venues': [{'name': 'Soldier Field', 'type': ..."
2,Dallas Cowboys vs. Buffalo Bills,https://www.ticketmaster.com/dallas-cowboys-vs...,{'public': {'startDateTime': '2019-04-18T00:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",NaN,NaN,"{'venues': [{'name': 'AT&T Stadium', 'type': '..."
3,Chicago Bears vs. Green Bay Packers,https://www.ticketmaster.com/chicago-bears-vs-...,{'public': {'startDateTime': '2019-04-18T01:30...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","[{'type': 'standard', 'currency': 'USD', 'min'...",{'info': 'There is an eight (8) ticket limit.'},"{'venues': [{'name': 'Soldier Field', 'type': ..."
4,Eagles,https://www.ticketmaster.com/eagles-las-vegas-...,{'public': {'startDateTime': '2019-04-12T17:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",NaN,{'info': 'Ticket limits vary per price level. ...,"{'venues': [{'name': 'MGM Grand Garden Arena',..."


In [427]:
df['genre'] = df.classifications.apply(lambda x: x[0]['genre']['name'])
df['subgenre'] = df.classifications.apply(lambda x: x[0]['subGenre']['name'])
df.drop(columns = ['classifications'],inplace = True)

In [428]:
#unpack presales ( need to merge)
sales_df = pd.io.json.json_normalize(df.sales)
sales_df.presales.fillna(0,inplace = True)
sales_df['presales'] = sales_df['presales'].apply(lambda x: 1 if x!= 0 else 0)
#drop na (not onsale yet) cant merge!!!
# sales_df.dropna(inplace = True)
sales_df.drop(columns=['public.startTBD'], inplace = True)
sales_df['public.startDateTime'] = pd.to_datetime(sales_df['public.startDateTime'])
sales_df['public.endDateTime'] = pd.to_datetime(sales_df['public.endDateTime'])
#need to merge
df.drop(columns=['sales'],inplace=True)

In [429]:
df = df.merge(sales_df, on=df.index)

In [430]:
#unpack price
df.priceRanges.fillna(1, inplace=True) # if no pricerange json in cell then event sold out
df['sold_out'] = df.priceRanges.apply(lambda x: x if x==1 else 0)
df['price_min'] = df.priceRanges.apply(lambda x: x[0]['min'] if x !=1 else np.nan)
df['price_max'] = df.priceRanges.apply(lambda x: x[0]['max'] if x !=1 else np.nan)
df.drop(columns=['priceRanges'],inplace=True)

In [431]:
import string

def ticket_limit(x):
    try:
        return x['info'].split('(', 1)[1].split(')')[0]
    except:
        return 'arb'
    
def is_numeric(s):
    if s[0] in string.digits:
        return True
    else: 
        False
    

In [432]:
#unpack ticketlimit
df.ticketLimit.fillna('arb',inplace=True)
df['max_tickets'] = df.ticketLimit.apply(lambda x: ticket_limit(x) if x !='arb' else x)
df['max_tickets'] = df.max_tickets.apply(lambda x: x if is_numeric(x) else 'arb')
df.max_tickets.replace('arb', np.nan, inplace=True)
df.drop(columns = ['ticketLimit'], inplace=True)

In [433]:
df.head()

,key_0,name,url,_embedded,genre,subgenre,presales,public.startDateTime,public.endDateTime,sold_out,price_min,price_max,max_tickets
0,0,New York Jets vs. Cleveland Browns,https://www.ticketmaster.com/new-york-jets-vs-...,"{'venues': [{'name': 'MetLife Stadium', 'type'...",Football,NFL,1,2019-04-22 14:00:00+00:00,2019-09-17 00:15:00+00:00,0,28.9,802.0,6
1,1,Chicago Bears vs. Los Angeles Chargers,https://www.ticketmaster.com/chicago-bears-vs-...,"{'venues': [{'name': 'Soldier Field', 'type': ...",Football,NFL,1,2019-04-18 01:30:00+00:00,2019-10-27 18:00:00+00:00,0,140.0,596.0,15
2,2,Dallas Cowboys vs. Buffalo Bills,https://www.ticketmaster.com/dallas-cowboys-vs...,"{'venues': [{'name': 'AT&T Stadium', 'type': '...",Football,NFL,1,2019-04-18 00:00:00+00:00,2019-11-28 22:30:00+00:00,1,NaN,NaN,NaN
3,3,Chicago Bears vs. Green Bay Packers,https://www.ticketmaster.com/chicago-bears-vs-...,"{'venues': [{'name': 'Soldier Field', 'type': ...",Football,NFL,1,2019-04-18 01:30:00+00:00,2019-09-06 01:20:00+00:00,0,185.0,1093.0,8
4,4,Eagles,https://www.ticketmaster.com/eagles-las-vegas-...,"{'venues': [{'name': 'MGM Grand Garden Arena',...",Rock,Pop,1,2019-04-12 17:00:00+00:00,2019-09-28 01:00:00+00:00,1,NaN,NaN,NaN


In [434]:
venue_df = pd.io.json.json_normalize(df['_embedded'].apply(lambda x: x['venues'][0]))

In [435]:
venue_df = venue_df[['name','postalCode','markets','city.name', 'state.name', 'state.stateCode', 'country.countryCode', 
                     'address.line1','location.longitude', 'location.latitude']]

In [436]:
venue_df.columns = ['venue_name', 'postalCode', 'markets', 'city.name', 'state.name',
       'state.stateCode', 'country.countryCode', 'address.line1',
       'location.longitude', 'location.latitude']

In [437]:
df = df.join(venue_df)

In [438]:
df.drop(columns = ['_embedded', 'key_0'], inplace=True)

In [445]:
def num_of_markets(x):
    try:
        return len(x)
    except:
        return x

In [446]:
df['num_markets'] = df.markets.apply(lambda x: num_of_markets(x))

In [447]:
df.drop(columns = ['markets'],inplace=True)

In [455]:
df.drop(columns = ['state'],inplace = True)

In [459]:
df.columns = ['event_name', 'url', 'genre', 'subgenre', 'is_presale', 'onsale_date','event_date', 'is_sold_out', 'price_min', 'price_max', 
              'max_tickets','venue_name', 'postalCode', 'city', 'state','country', 'address', 'longitude', 'latitude','num_markets']

In [460]:
df

,event_name,url,genre,subgenre,is_presale,onsale_date,event_date,is_sold_out,price_min,price_max,max_tickets,venue_name,postalCode,city,state,country,address,longitude,latitude,num_markets
0,New York Jets vs. Cleveland Browns,https://www.ticketmaster.com/new-york-jets-vs-...,Football,NFL,1,2019-04-22 14:00:00+00:00,2019-09-17 00:15:00+00:00,0,28.90,802.00,6,MetLife Stadium,07073,East Rutherford,NJ,US,One MetLife Stadium Drive,-74.074493,40.81359,3.0
1,Chicago Bears vs. Los Angeles Chargers,https://www.ticketmaster.com/chicago-bears-vs-...,Football,NFL,1,2019-04-18 01:30:00+00:00,2019-10-27 18:00:00+00:00,0,140.00,596.00,15,Soldier Field,60605,Chicago,IL,US,1410 S. Museum Campus Drive,-87.61682143,41.86205404,2.0
2,Dallas Cowboys vs. Buffalo Bills,https://www.ticketmaster.com/dallas-cowboys-vs...,Football,NFL,1,2019-04-18 00:00:00+00:00,2019-11-28 22:30:00+00:00,1,NaN,NaN,NaN,AT&T Stadium,76011,Arlington,TX,US,One AT&T Way,-97.09345578,32.74790427,2.0
3,Chicago Bears vs. Green Bay Packers,https://www.ticketmaster.com/chicago-bears-vs-...,Football,NFL,1,2019-04-18 01:30:00+00:00,2019-09-06 01:20:00+00:00,0,185.00,1093.00,8,Soldier Field,60605,Chicago,IL,US,1410 S. Museum Campus Drive,-87.61682143,41.86205404,2.0
4,Eagles,https://www.ticketmaster.com/eagles-las-vegas-...,Rock,Pop,1,2019-04-12 17:00:00+00:00,2019-09-28 01:00:00+00:00,1,NaN,NaN,NaN,MGM Grand Garden Arena,89109,Las Vegas,NV,US,3799 Las Vegas Blvd South,-115.1694962,36.1023376,1.0
5,2020 CMA Fest,https://www.ticketmaster.com/2020-cma-fest-nas...,Country,Country,1,2019-08-02 15:00:00+00:00,2020-06-08 01:00:00+00:00,0,235.05,466.50,8,Nissan Stadium,37213,Nashville,TN,US,One Titans Way,-86.771413,36.166208,1.0
6,Chicago Bears vs. Carolina Panthers,https://www.ticketmaster.com/chicago-bears-vs-...,Football,NFL,1,2019-04-18 01:30:00+00:00,2019-08-09 01:00:00+00:00,0,41.00,330.00,20,Soldier Field,60605,Chicago,IL,US,1410 S. Museum Campus Drive,-87.61682143,41.86205404,2.0
7,KoRn & Alice In Chains,http://concerts.livenation.com/event/1D0056554...,Rock,Alternative Rock,1,2019-03-01 15:00:00+00:00,2019-08-06 22:00:00+00:00,0,25.00,142.50,NaN,Northwell Health at Jones Beach Theater,11793,Wantagh,NY,US,895 Bay Parkway,-73.5023618,40.6014755,2.0
8,Chivas Guadalajara v. Club America,https://www.ticketmaster.com/chivas-guadalajar...,Soccer,Soccer,1,2019-07-17 17:00:00+00:00,2019-09-08 05:00:00+00:00,0,45.00,300.00,NaN,Soldier Field,60605,Chicago,IL,US,1410 S. Museum Campus Drive,-87.61682143,41.86205404,2.0
9,Jeezy TM 104 Concert,http://concerts.livenation.com/event/0E0056FDE...,Hip-Hop/Rap,French Rap,1,2019-08-02 16:00:00+00:00,2019-08-24 01:00:00+00:00,0,63.00,103.00,NaN,Coca-Cola Roxy,30339,Atlanta,GA,US,800 Battery Ave SE #500,-84.469717,33.89024,1.0
